## Install Dependencies

In [2]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard fbe67e465375231474a2ad80a4389efc77ecff99

Cloning into 'yolov5'...
remote: Enumerating objects: 16793, done.
remote: Counting objects: 100% (334/334), done.
remote: Compressing objects: 100% (252/252), done.
remote: Total 16793 (delta 166), reused 208 (delta 82), pack-reused 16459
Receiving objects: 100% (16793/16793), 15.67 MiB | 5.69 MiB/s, done.
Resolving deltas: 100% (11469/11469), done.
/Users/asmit/Programming/Projects/clash_of_clans/models/yolov5/yolov5
HEAD is now at fbe67e46 Fix `OMP_NUM_THREADS=1` for macOS (#8624)


In [3]:
# Install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

You should consider upgrading via the '/Users/asmit/Programming/Projects/clash_of_clans/.venv/bin/python -m pip install --upgrade pip' command.
Setup complete. Using torch 2.3.1 CPU


## Define Model Architecture

In [4]:
# Define number of classes based on YAML
import yaml

# Define the location of the data and model for later
data_location = "/Users/asmit/Programming/Projects/clash_of_clans/data/preprocessed_data"
model_location = "/Users/asmit/Programming/Projects/clash_of_clans/models/yolov5/models"

with open(data_location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])
    print(f"Number of classes: {num_classes}")

Number of classes: 16


In [5]:
# Using default model configuration
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [6]:
%%writetemplate {model_location}/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

## Train Model

In [14]:
%%time
%cd /Users/asmit/Programming/Projects/clash_of_clans/models/yolov5/

"""Train model with following params:
    - Image size: 1920x1920
    - Batch size: 16
    - Epochs: 1
    - Data: data_location (specified above)
    - Config: model_location (specified above)
    - Weights: '' (Using default)
    - Name: yolov5s_results
    - Cache
"""
!python train.py --img 1920 --batch 16 --epochs 1 --data {data_location}/data.yaml --cfg {model_location}/custom_yolov5s.yaml --weights '' --name yolov5s_results  --cache

/Users/asmit/Programming/Projects/clash_of_clans/models/yolov5
train: weights=, cfg=/Users/asmit/Programming/Projects/clash_of_clans/models/yolov5/models/custom_yolov5s.yaml, data=/Users/asmit/Programming/Projects/clash_of_clans/data/preprocessed_data/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=1, batch_size=16, imgsz=1920, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5s_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 699 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-306-gfbe67e46 Python-3.11.4 torch-2.3.1 CPU

hyp